In [2]:
#importing data

%pylab inline
import neurosynth as ns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
#only once
#ns.dataset.download(path='.', unpack=True)

Populating the interactive namespace from numpy and matplotlib


In [3]:
#long runtime 
#from neurosynth.base.dataset import Dataset
#dataset = ns.base.dataset.Dataset('database.txt');

In [4]:
#medium runtime
#dataset.add_features('features.txt');

In [5]:
#ids1 = dataset.get_studies(expression='alzheimer', frequency_threshold=0.001)

In [6]:
#data is saved in studies and 
df= pd.read_table('database.txt')
#t= mni_space['MNI'];
mni_space= df[df['space'].str.contains("MNI")];
x_vals = list(mni_space['x']);
y_vals = list(mni_space['y']);
z_vals = list(mni_space['z']);
study_ids = list(mni_space['id']);
unique_ids = list(set(study_ids)); #removing duplicates

term_table= pd.read_table('features.txt');
mni_term_table = term_table.loc[term_table['pmid'].isin(unique_ids)];
terms = term_table.columns.values;

id_x_features = np.array(mni_term_table['pmid']);

database_labels = df.columns.values;
print database_labels

print mni_term_table.shape

['id' 'doi' 'x' 'y' 'z' 'space' 'peak_id' 'table_id' 'table_num' 'title'
 'authors' 'year' 'journal']
(7669, 3407)


In [7]:
#print id_x_features[9243]
#print mni_term_table#.iloc[9243][1:]

In [8]:
def initStudyDict():
    '''Creates a dictionary with Keys = study ids and Vals = tuples of x,y,z locations in NMI space'''
    
    '''Add parameter to smear xyz locations'''
    ids = {};
    c = 0;
    for i in study_ids:
        #print i
        if i not in ids:
            ids[i] = [(x_vals[c],y_vals[c],z_vals[c])];
            c +=1;
        elif (x_vals[c],y_vals[c],z_vals[c]) in ids[i]:
            c+=1
        else:
            ids[i].append((x_vals[c],y_vals[c],z_vals[c]))
    return ids     
id_mat = initStudyDict();

In [9]:
#bookeeping methods

def isTerm(term):
    '''Checks if this term is in the neurosynth database'''
    if term in mni_term_table:
        return True
    else:
        return False

def isLocation(coords):
    '''Checks if coordinate set (x,y,z) is mentioned in any studies'''
    if np.floor(coords[0]) in np.floor(x_vals):
        #ind = x_vals.index(coords[0]);
        
        for xind in xrange(len(x_vals)):
            if coords[0] == x_vals[xind]:
                if np.floor(y_vals[xind]) == np.floor(coords[1]): 
                    #print '2'
                    if np.floor(z_vals[xind]) == np.floor(coords[2]):
                        return True
        else:
            return False
    else: 
        return False
    
def isID(ID):
    '''Checks if ID is an ID with NMI coordinates'''
    if ID in id_mat:
        return True
    else:
        return False
    
    
def CoordtoIDs(coord):
    '''Uses the study dictionary above to find study ids from x,y,z coordinates'''
    if isLocation(coord):
        IDs = [];
        for i, coords in id_mat.items():
            if coord in coords:
                IDs.append(i)
        return IDs
    else:
        return "These coordinates don't match any studies"

def IDtoTerms(ID):
    '''Finds all of the term heat values of a given ID'''
    if isID(ID):
        ind= int(np.squeeze(np.where(id_x_features == ID)))
        return list(mni_term_table.iloc[ind][1:])
    else:
        return 'Not an ID of a study in NMI space';

def IDtoCoords(ID):
    '''Finds coordinates associated with a given study ID'''
    if isID(ID):
        return id_mat[ID]
    else:
        return 'Not an ID of a study in NMI space'


def CoordtoTerms(coord):
    '''Returns the vector of term heats for a given (x,y,z) coordinate set. 
    If there are multiple studies that mention the same coordinates, the average is taken.'''
    if isLocation(coord):
        ids = CoordtoIDs(coord)
        if len(ids) == 1:
            return IDtoTerms(ids[0])
        else: 
            temp = np.zeros((len(ids),3406));
            for i in xrange(len(ids)):
                temp[i,:] = IDtoTerms(ids[i]);
            return list(mean(temp,0))
    else:
        return 'not valid location'

    
def TermtoIDs(term,thresh):
    '''Matches a term to the IDs of studies that use that term above a given threshold'''
    if isTerm(term):
        term_all_ids = np.array(mni_term_table[term]);
        id_inds = squeeze(np.where(term_all_ids>thresh));
        return id_x_features[id_inds]
    else:
        return 'This is not a valid term'
    
def TermtoCoords(term,thresh):
    '''Finds the coordinates that are associated with a given term up to a given threshold'''
    ids = TermtoIDs(term,thresh);
    coords = [IDtoCoords(i) for i in ids]
    return coords


#print TermtoIDs('schizophrenia',0.5)
c = TermtoIDs('vision',0.3)
print c

[16337921 20692350]


In [10]:
#distance methods

def calcDistance(coord1,coord2):
    '''Calculates Euclidian distance between two coordinates'''
    dist = np.sqrt((coord1[0]-coord2[0])**2 + (coord1[1]-coord2[1])**2 + (coord1[2]-coord2[2])**2);
    return dist

def findNeighbors(coord,dist):
    neighbors = [];
    for x in xrange(int(coord[0]-(dist+1)),int(coord[0]+(dist+1))):
        for y in xrange(int(coord[1]-(dist+1)),int(coord[1]+(dist+1))):
            for z in xrange(int(coord[2]-(dist+1)),int(coord[2]+(dist+1))):
                if calcDistance(coord,(x,y,z)) == dist:
                    neighbors.append((float(x),float(y),float(z)))    
    return neighbors
  
def sphere(coord,maxDist):
    '''Finds every point in a sphere around a given point of radius maxDist'''
    ind_sphere = [];
    for d in xrange(maxDist):
        ind_sphere.append(findNeighbors(coord,d));
    return ind_sphere
    
    
    
def assignWeights(ind_sphere,weight = 2):
    '''Assigns weights to a point sphere produced by the sphere method'''
    '''weight must be >1'''
    weight_vector = np.ones((1,len(ind_sphere)));  
    for layer in xrange(1,len(ind_sphere)):
        weight_vector[0,layer] = 1/(float(layer)*weight)
    
    return weight_vector

In [39]:
#estimating term weights of unknown location

def termVectorOfUnknownPoint(coord,maxDist):   
    '''Estimates the terms of an unknown point by drawing from known points around it using a sphere of radius maxDist'''
    if isLocation(coord) == True:
        return CoordtoTerms(coord)
    else:
        ind_sphere = sphere(coord,maxDist);
        weight_vect = assignWeights(ind_sphere);
        termVect = [];
        for layer in xrange(len(ind_sphere)):
            for c in ind_sphere[layer]:
                #print c
                if isLocation(c) == True:
                    print 'found a nearby point'
                    print c
                    if ~np.isnan(np.sum(CoordtoTerms(c))):
                        temp = CoordtoTerms(c);
                        termVect.append([t*weight_vect[0,layer] for t in temp])
                        print weight_vect[0,layer]
        
        if len(termVect)>1:
            #Need a better way to normalize
            termVect = np.sum(termVect,0);
            
        return termVect
    
#print termVectorOfUnknownPoint((-2.0, -34.0, -46.0),10)
tV = termVectorOfUnknownPoint((-2.0, -98.0, -6.0),8);
print len(tV)

found a nearby point
(-2.0, -98.0, -8.0)
0.25
found a nearby point
(-2.0, -98.0, -4.0)
found a nearby point
(-6.0, -98.0, -6.0)
0.125
found a nearby point
(-2.0, -98.0, -10.0)
0.125
found a nearby point
(-2.0, -95.0, -2.0)
0.1
found a nearby point
(-8.0, -98.0, -6.0)
0.0833333333333
found a nearby point
(-6.0, -96.0, -10.0)
0.0833333333333
found a nearby point
(-6.0, -96.0, -2.0)
found a nearby point
(-6.0, -94.0, -4.0)
0.0833333333333
found a nearby point
(-4.0, -94.0, -10.0)
found a nearby point
(-4.0, -94.0, -2.0)
found a nearby point
(-2.0, -98.0, 0.0)
0.0833333333333
found a nearby point
(0.0, -94.0, -2.0)
0.0833333333333
found a nearby point
(2.0, -94.0, -4.0)
0.0833333333333
found a nearby point
(-9.0, -98.0, -6.0)
found a nearby point
(-5.0, -92.0, -4.0)
3406


In [38]:
print sum(tV,0)

[ 0.  0.  0. ...,  0.  0.  0.]


In [151]:
print x_vals[0]
print y_vals[0]
print z_vals[0]

-2.0
-98.0
-8.0


In [10]:
#ns.network.coactivation(dataset, [[0, 20, 28]], threshold=0.1, output_dir='seed_test', prefix='acc_seed')

True


-34.0